In [ ]:
from bs4 import BeautifulSoup
import json
import requests
import re

In [6]:
urls = ["https://www.mosdac.gov.in/saral-references", "https://www.mosdac.gov.in/oceansat3-references", "https://www.mosdac.gov.in/insat-3s-references"]

In [7]:
def process_data(docs,h1):
        
        all_data = []

        for p in docs:
            a_tag = p.find("a")
            span_tag = p.find("span")

            if not a_tag or not span_tag:
                continue  # skip broken entries

            url = "https://www.mosdac.gov.in" + a_tag["href"]
            title = a_tag.get_text(strip=True)

            # normalize and extract info
            info_text = span_tag.get_text(strip=True)

            size_match = re.search(r"Size\s*:\s*([\d.]+\s*\w+)", info_text, re.I)
            format_match = re.search(r"Format\s*:\s*(\w+)", info_text, re.I)
            tool_match = re.search(r"Tool\s*:\s*(\w+)", info_text, re.I)

            size = size_match.group(1) if size_match else None
            format_ = format_match.group(1) if format_match else None
            tool = tool_match.group(1) if tool_match else None

            data = {
                "title": title,
                "url": url,
                "size": size,
                "format": format_,
                "tool": tool
            }

            all_data.append(data)

        with open(f"C:/Users/WELCOME/Desktop/mosdac/missions_data/{h1}.json", "w", encoding="utf-8") as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)


In [ ]:
for url in urls:
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {url}")
        print("its likely it doesnt have a document section.")
        continue

    print(f"sucessful: {response.status_code}: {url}")
    
    soup = BeautifulSoup(response.text, "html.parser")
    h1 = soup.find("h1").get_text(strip=True).replace(" ","-")
    docs = soup.find("div", class_="field-item even").find_all("p")
    process_data(docs,h1)
    